# 1. Import Dependencies

In [1]:
!pip install opencv-python

In [2]:
# Import opencv
import cv2

# Import uuid
import uuid

# Import Operating System
import os

# Import time
import time

#Import Numpy
import numpy as np

import shutil

import random as rd

# 2. Define Images to Collect

In [3]:
#Specify What You Want Labelled As Well As Empty Train & Test Folders For Later
labels = ['Mask', 'NoMask']
number_imgs = 5

# 3. Setup Folders 

In [4]:
#PATHS
FOLDER_PATH = os.getcwd()
TF_PATH = os.path.join(FOLDER_PATH,'Tensorflow','workspace','images', 'collectedimages')
IMG_PATH = os.path.join(FOLDER_PATH, 'Tensorflow','workspace','images')

DOWNLOAD_IMG_PATH = os.path.join(FOLDER_PATH, 'Images')

folders = ["Train", "Test"]

In [5]:
print(FOLDER_PATH)

D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection


In [6]:
#Create Folders for Each Label
if not os.path.exists(TF_PATH):
    if os.name == 'posix':
        !mkdir -p {TF_PATH}
    if os.name == 'nt':
        !mkdir {TF_PATH}

#Create Train & Test Folders
os.chdir(IMG_PATH)
for folder in folders:
    path = os.path.join(IMG_PATH, folder)
    if not os.path.exists(path):
        !mkdir {path}
        print("Folder Created For {}....".format(folder))
    else:
        print("Folder Already Created For {}....".format(folder))
        
#Create Folders For Custom Labels
for label in labels:
    path = os.path.join(TF_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}
        print("Folder Created For {}....".format(label))
    else:
        print("Folder Already Created For {}....".format(label))

Folder Created For Train....
Folder Created For Test....
Folder Created For Mask....
Folder Created For NoMask....


# 4. Capture Images On Camera (Optional)

In [ ]:
save_Files_PATH = os.path.join(IMG_PATH,'collectedimages')

for label in labels:
    cap = cv2.VideoCapture(0)
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    for imgnum in range(number_imgs):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(save_Files_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(2)

        if cv2.waitKey(1) == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

# 4. Copy Images From Other Folder

In [7]:
for label in labels:
    src = os.path.join(DOWNLOAD_IMG_PATH, label)
    print(DOWNLOAD_IMG_PATH)
    dest = os.path.join(TF_PATH, label)
    os.chdir(src)
    files = os.listdir(src)
    if len(os.listdir(dest))== 0:
        for file in files:
            shutil.copy(file, dest)
        print("Files Copied To Destination Folder....")
    else:
        print("Files Already Copied From {} Folder...".format(label))

D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection\Images
Files Copied To Destination Folder....
D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection\Images
Files Copied To Destination Folder....


# 5. Renaming Images

In [8]:
#Rename Images to <label>.num.jpg

for label in labels:
    DST_label_PATH = os.path.join(TF_PATH, label)
    os.chdir(DST_label_PATH)
    DIR = os.listdir(DST_label_PATH)
    for count, file_name in enumerate(DIR):
        src = DST_label_PATH + '\\' + file_name
        dst = DST_label_PATH + '\\' + label + "_" + str(count+1) + ".jpg"
        os.rename(src, dst)
print("Images Successfully Renamed...")

Images Successfully Renamed...


# 6. Import Image Labelling 

In [9]:
!pip install --upgrade pyqt5 lxml

In [10]:
LABEL_IMG_PATH = os.path.join(FOLDER_PATH,'Tensorflow','labelImg')

In [11]:
#Create a Folder For Image Labeling 
if not os.path.exists(LABEL_IMG_PATH):
    !mkdir {LABEL_IMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABEL_IMG_PATH}
    print("Folder Created and Repo Copied...")
else:
    print("Folder Already Created and Repo Copied....")

Folder Created and Repo Copied...


Cloning into 'D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection\Tensorflow\labelImg'...


In [12]:
if os.name == 'posix':
    !cd {LABEL_IMG_PATH} && make qt5py3
if os.name =='nt':
    !cd {LABEL_IMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

# 7. Open Label Image Probram & Label Images

In [13]:
#Open Label Image Program
!cd {LABEL_IMG_PATH} && python labelImg.py

Image:D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection\Tensorflow\workspace\images\collectedimages\Mask\Mask_1.jpg -> Annotation:D:/Jeremy/Desktop/Tensorflow/CustomObjectDetection/Tensorflow/workspace/images/collectedimages/Mask/Mask_1.xml
Image:D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection\Tensorflow\workspace\images\collectedimages\Mask\Mask_2.jpg -> Annotation:D:/Jeremy/Desktop/Tensorflow/CustomObjectDetection/Tensorflow/workspace/images/collectedimages/Mask/Mask_2.xml
Cancel creation.
Image:D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection\Tensorflow\workspace\images\collectedimages\Mask\Mask_3.jpg -> Annotation:D:/Jeremy/Desktop/Tensorflow/CustomObjectDetection/Tensorflow/workspace/images/collectedimages/Mask/Mask_3.xml
Image:D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection\Tensorflow\workspace\images\collectedimages\Mask\Mask_4.jpg -> Annotation:D:/Jeremy/Desktop/Tensorflow/CustomObjectDetection/Tensorflow/workspace/images/collectedimages/Mask/Mask_4.xml
Image:D:\Jeremy

qt.gui.icc: fromIccProfile: failed minimal tag size sanity
qt.gui.icc: fromIccProfile: failed minimal tag size sanity



Image:D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection\Tensorflow\workspace\images\collectedimages\Mask\Mask_7.jpg -> Annotation:D:/Jeremy/Desktop/Tensorflow/CustomObjectDetection/Tensorflow/workspace/images/collectedimages/Mask/Mask_7.xml
Image:D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection\Tensorflow\workspace\images\collectedimages\Mask\Mask_8.jpg -> Annotation:D:/Jeremy/Desktop/Tensorflow/CustomObjectDetection/Tensorflow/workspace/images/collectedimages/Mask/Mask_8.xml
Image:D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection\Tensorflow\workspace\images\collectedimages\Mask\Mask_9.jpg -> Annotation:D:/Jeremy/Desktop/Tensorflow/CustomObjectDetection/Tensorflow/workspace/images/collectedimages/Mask/Mask_9.xml
Image:D:\Jeremy\Desktop\Tensorflow\CustomObjectDetection\Tensorflow\workspace\images\collectedimages\Mask\Mask_10.jpg -> Annotation:D:/Jeremy/Desktop/Tensorflow/CustomObjectDetection/Tensorflow/workspace/images/collectedimages/Mask/Mask_10.xml
Image:D:\Jeremy\Desktop\Tenso

# 8. Splitting Images Into Training & Testing

In [14]:
#Make a function to move files to Train folder

def move_file_Train(src_path):
    dst = os.path.join(IMG_PATH, 'Train')
    src = src_path
    shutil.move(src, dst)

In [15]:
#Make a function to move files to Test folder

def move_file_Test(src_path):
    dst = os.path.join(IMG_PATH, 'Test')
    src = src_path
    shutil.move(src, dst)

In [16]:
def rand_num_gen(length_dir):
    randomNum = rd.randrange(1, length_dir)
    return randomNum

In [29]:
#Training Ratio Split Into Train Folder
TRAIN_RATIO = 0.7
randList = []
for label in labels:
    DST_label_PATH = os.path.join(TF_PATH, label)
    os.chdir(DST_label_PATH)
    DIR = os.listdir(DST_label_PATH)
    LEN_DIR = len(DIR)/2
    NUM_IMG_SPLIT = LEN_DIR*TRAIN_RATIO
    print("Number of Images For Training Taken From {}: {}".format(label, int(NUM_IMG_SPLIT)))
    for x in range(int(NUM_IMG_SPLIT)):
        randNum = rand_num_gen(LEN_DIR)
        if len(randList) != int(NUM_IMG_SPLIT) and randNum not in randList:
            randList.append(randNum)
        else:
            while (randNum in randList):
                randNum = rand_num_gen(LEN_DIR)
            if len(randList) != int(NUM_IMG_SPLIT) and randNum not in randList:
                randList.append(randNum)
        
        IMG_SELECT = os.path.join(DST_label_PATH, label + "_" + str(randList[x]) + ".jpg")
        LABELIMG_SLECT = os.path.join(DST_label_PATH, label + "_" + str(randList[x]) + ".xml")
        #print("File {}_{}.jpg Moved To Train Folder".format(label, randList[x]))
        #print("File {}_{}.xml Moved To Train Folder".format(label, randList[x]))
        move_file_Train(IMG_SELECT)  
        move_file_Train(LABELIMG_SLECT)
print("Files Successfully Moved To Train Folder....")

Number of Images For Training Taken From Mask: 16
Number of Images For Training Taken From NoMask: 20
20.299999999999997 Files Successfully Moved To Train Folder


In [30]:
#Remainder Images Moved Into Test Folder

for label in labels:
    DST_label_PATH = os.path.join(TF_PATH, label)
    os.chdir(DST_label_PATH)
    DIR = os.listdir(DST_label_PATH)
    print("Number of Images For Training Taken From {}: {}".format(label, int(NUM_IMG_SPLIT))) 
    for file_name in DIR:
        file = os.path.join(DST_label_PATH, file_name)
        move_file_Test(file)
        #print("File {} Moved To Test Folder".format(file_name))
        
print("Files Successfully Moved To Test Folder....")

Number of Images For Training Taken From Mask: 20
Number of Images For Training Taken From NoMask: 20
Files Successfully Moved To Test Folder....


# Continue To 3. Training & Testing Notebook